<a href="https://colab.research.google.com/github/ahmedynah/Epsilon-AI/blob/main/Web_Scraping_Lecture_10_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Task 1
---------
#####Scrap the USD To EGP Exchange rate from this website
#####https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html
#####and then use it to make a software that takes amount of USD Dollars from the user and calculate how much will it cost in EGP.

In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import time

In [ ]:
amount = float(input("amount in USD: "))
url = 'https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html'
res = requests.get(url)
soap = BeautifulSoup(res.text, 'html.parser')

amount in USD: 5


In [ ]:
rate = float(soap.find('', attrs = {'id':"shd2b;"}).get_text())
print(f"{amount} USD = {amount * rate} EGP")

5.0 USD = 78.5005 EGP


Tassk 2
---------
#####Scrap the weather temperature, humidity, visibility, air pressure, wind speed, datetime data from this website
#####https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt
#####for 10 times one read in a minute (so basicly the process takes 10 minutes)and put them into a CSV file

In [ ]:
def get_weather(url):
  
  res = requests.get(url)
  
  soap = BeautifulSoup(res.text, 'html.parser')
  temp = soap.find('div', attrs={"class": "temp"}).get_text()
  stats = soap.find('div', attrs={"class": "stats"}).get_text().replace("Rel. ","")
  
  stats = stats.replace('Few Clouds at 760m',"")
  stats = stats.replace(" |","").splitlines()
  
  stats_headers, stats_values = [header.strip()for header in stats[0::2]], [value.strip() for value in stats[1::2]]
  
  stats_headers.insert(0,"temp")
  stats_values.insert(0,temp)

  return stats_headers, stats_values

In [ ]:
def repeate_every_x_min_for_10_times(x):
    with open("weather.csv", "w") as f:
      url = 'https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt'
      headers, values = get_weather(url)
      writer = csv.writer(f)
      writer.writerow(headers)
      for _ in range(10):
         url = 'https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt'
         headers, values = get_weather(url)
         writer.writerow(values)
    time.sleep(x * 60)

In [ ]:
repeate_every_x_min_for_10_times(1)

#Task 3
---------
#####Scrap the books (name, price, rate) for each category and put them into a CSV file
#####https://books.toscrape.com/

In [32]:
url = 'https://books.toscrape.com/catalogue/category/books_1/index.html'

In [33]:
res = requests.get(url)

In [34]:
soap = BeautifulSoup(res.text, 'html.parser')
index = soap.find('a', attrs={'href':'index.html'})
cat_trav = index.find_next_sibling('ul')
cat_trav2 = cat_trav.findAll('a')
categories = [e.get_text(strip = True) for e in cat_trav2]
print(categories)
links_of_each_cat = cat_trav.findAll('li')
links_of_each_cat = [link.find('a').get('href').replace("..",'https://books.toscrape.com/catalogue/category')
for link in links_of_each_cat]



['Travel', 'Mystery', 'Historical Fiction', 'Sequential Art', 'Classics', 'Philosophy', 'Romance', 'Womens Fiction', 'Fiction', 'Childrens', 'Religion', 'Nonfiction', 'Music', 'Default', 'Science Fiction', 'Sports and Games', 'Add a comment', 'Fantasy', 'New Adult', 'Young Adult', 'Science', 'Poetry', 'Paranormal', 'Art', 'Psychology', 'Autobiography', 'Parenting', 'Adult Fiction', 'Humor', 'Horror', 'History', 'Food and Drink', 'Christian Fiction', 'Business', 'Biography', 'Thriller', 'Contemporary', 'Spirituality', 'Academic', 'Self Help', 'Historical', 'Christian', 'Suspense', 'Short Stories', 'Novels', 'Health', 'Politics', 'Cultural', 'Erotica', 'Crime']


In [36]:
def write_in_csv():
  with open("books.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Rate', "Price", "Link", "Category"])
    i = 0
    for cat_link in links_of_each_cat:
      books = traverse_pages(cat_link)
      for book in books:
        book.append(categories[i])
        writer.writerow(book)
      i += 1

In [35]:
def get_books_info(link):
  res = requests.get(link)
  soap = BeautifulSoup(res.text,"html.parser")
  books = soap.findAll('article', attrs={'class':'product_pod'})
  books_info = [
     [book.find('h3').find('a').get('title')
     ,book.find('p').get('class')[1] + " stars"
     ,book.find('p',attrs={'class':'price_color'}).get_text().replace("Â","")
     ,book.find('h3').find('a').get('href').replace("../../..","https://books.toscrape.com/catalogue")]
  for book in books
  ]
  return books_info

In [162]:
def are_there_multiple_pages(link):
  res = requests.get(link)
  soap = BeautifulSoup(res.text,'html.parser')
  multiple_pages = soap.find('ul',attrs={'class':'pager'})
  if multiple_pages != None:
    if(multiple_pages.findAll('li')[0].get_text().strip()=="previous"):
      number_of_pages = multiple_pages.findAll('li')[1].get_text()
      number_of_pages = number_of_pages.strip().split()[-1]
    else:
      number_of_pages = multiple_pages.findAll('li')[0].get_text()
      number_of_pages = number_of_pages.strip().split()[-1]
    return True, int(number_of_pages), multiple_pages
  else:
    return False, 0, 0

In [163]:
def get_next_page(link, pager, i):
  next_page = pager.find('li', attrs={'class':'next'}).find('a').get('href')
  if(link.find("index.html") == -1):
    link = link.replace(f"page-{i}.html",next_page)
    # print(f"{i} = in1 and next page = {next_page} and link = {link}")
  else:
    link = link.replace("index.html",next_page)
    # print(f"{i} = in2 and next page = {next_page} and link = {link}")
  return link

In [168]:
def traverse_pages(link):
  
  all_books = []
  check, no_pages, pager = are_there_multiple_pages(link)
  if check:    
    for i in range(no_pages):
      _, _, pager = are_there_multiple_pages(link)
      # print(link, "\n", i+1," ", no_pages, " ",pager)
      all_books.extend(get_books_info(link))
      if(i+1 != no_pages):
        link = get_next_page(link, pager, i+1)
      # res = requests.get(link)
  else:
    all_books.extend(get_books_info(link))
  # print(all_books)
  return all_books

In [169]:
traverse_pages(links_of_each_cat[3])

[["Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
  'Five stars',
  '£52.29',
  'https://books.toscrape.com/catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html'],
 ['Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRoNiCLE #2)',
  'One stars',
  '£16.28',
  'https://books.toscrape.com/catalogue/tsubasa-world-chronicle-2-tsubasa-world-chronicle-2_949/index.html'],
 ['This One Summer',
  'Four stars',
  '£19.49',
  'https://books.toscrape.com/catalogue/this-one-summer_947/index.html'],
 ['The Nameless City (The Nameless City #1)',
  'Four stars',
  '£38.16',
  'https://books.toscrape.com/catalogue/the-nameless-city-the-nameless-city-1_940/index.html'],
 ['Saga, Volume 5 (Saga (Collected Editions) #5)',
  'Two stars',
  '£51.04',
  'https://books.toscrape.com/catalogue/saga-volume-5-saga-collected-editions-5_923/index.html'],
 ['Rat Queens, Vol. 3: Demons (Rat Queens (Collected Editions) #11-15)',
  'Three stars',
  '£50.40',
  'https://books.toscrape.com/ca

In [170]:
write_in_csv()